In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from xgboost import XGBRegressor

pd.set_option('mode.chained_assignment',  None)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_row', 1300)


origin_train = pd.read_csv('train.csv')
origin_test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')


In [40]:
origin_train.columns = ["Date", "Day", "Total", "Leave", "Business-Trip", "Overtime-Work",
                "Home-Working", "Breakfast-Menu", "Lunch-Menu", "Dinner-Menu", "Lunch-Num", "Dinner-Num"]
origin_test.columns = ["Date", "Day", "Total", "Leave", "Business-Trip", "Overtime-Work",
                "Home-Working", "Breakfast-Menu", "Lunch-Menu", "Dinner-Menu"]


train = origin_train.copy()
test = origin_test.copy()
day_miss = train[train['Date'].str.contains('2018-06-01')].index
train.loc[day_miss[0], 'Day'] = '금'
train['Lunch-Num-Ratio'] = train['Lunch-Num'] / train['Total']
train['Dinner-Num-Ratio'] = train['Dinner-Num'] / train['Total']

def preprocess(origin):
    df = origin.copy()

    if 'Dinner-Num' in df.columns:
        df = df[df['Dinner-Num'] != 0]

    df.index = pd.to_datetime(df['Date'])
    df = df.drop(['Date'], axis=1)

    df['Log-Total'] = np.log(df['Total'])
    df['Log-Total'] *= df['Total'].min() / df['Log-Total'].min()

    df['Work-Num'] = df['Total'] - df['Leave'] - df['Business-Trip']
    df['Remain-Num'] = df['Total'] - df['Leave'] - df['Business-Trip'] - df['Home-Working']

    df['Log-Work-Num'] = df['Log-Total'] - df['Leave'] - df['Business-Trip']
    df['Log-Remain-Num'] = df['Log-Total'] - df['Leave'] - df['Business-Trip'] - df['Home-Working']

    df['Leave/Total'] = df['Leave'] / df['Total']
    df['Business/Total'] = df['Business-Trip'] / df['Total']
    df['Overtime/Total'] = df['Overtime-Work'] / df['Total']
    df['Overtime/Work'] = df['Overtime-Work'] / df['Work-Num']
    df['Overtime/Remain'] = df['Overtime-Work'] / df['Remain-Num']

    df['Log-Leave/Total'] = df['Leave'] / df['Log-Total']
    df['Log-Business/Total'] = df['Business-Trip'] / df['Log-Total']
    df['Log-Overtime/Total'] = df['Overtime-Work'] / df['Log-Total']
    df['Log-Overtime/Work'] = df['Overtime-Work'] / df['Log-Work-Num']
    df['Log-Overtime/Remain'] = df['Overtime-Work'] / df['Log-Remain-Num']

    df['Log-Overtime'] = df['Log-Total'] / (df['Log-Total'] + df['Overtime-Work'])
    df['Log-Remain-Over'] = df['Log-Remain-Num'] / (df['Log-Remain-Num'] + df['Overtime-Work'])

    df['Day'] = df['Day'].map({'월': 0, '화': 1, '수': 2, '목': 3, '금': 4})

    df = df.drop(columns=['Breakfast-Menu', 'Lunch-Menu', 'Dinner-Menu'])

    df['Week'] = df.index.week

    return df


train = preprocess(origin_train)
test = preprocess(origin_test)


<ipython-input-40-1f9863d2f9fa>:51: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  df['Week'] = df.index.week


In [41]:
#이번주 출근 수
left = []
temp = 5
a = 1
for i in train['Week']:
    if i == temp:
        left.append(a)
        a+=1
        temp = i
    else:
        a = 1
        left.append(a)
        a+=1
        temp = i

train['chul'] = left
trainx = pd.get_dummies(train['chul'])
train = pd.concat([train,trainx],axis=1)
train.drop(['chul'], inplace=True , axis=1)
left = []
temp = 4
a = 1
for i in test['Week']:
    if i == temp:
        left.append(a)
        a+=1
        temp = i
    else:
        a = 1
        left.append(a)
        a+=1
        temp = i

test['chul'] = left

testx = pd.get_dummies(test['chul'])
test = pd.concat([test,testx],axis=1)
test.drop(['chul'], inplace=True , axis=1)

train_X = train.drop(columns=['Lunch-Num', 'Dinner-Num'])
train_Y = train['Lunch-Num']
test_X = test

kfolds = KFold(n_splits=10, shuffle=True, random_state=142)
pred_lunch = []
train_lunch = []

for n_fold, (trn_idx, val_idx) in enumerate(kfolds.split(train)):
    train_X, train_Y = train.iloc[trn_idx].drop(columns=['Lunch-Num', 'Dinner-Num']), train.iloc[trn_idx]['Lunch-Num']
    valid_X, valid_Y = train.iloc[val_idx].drop(columns=['Lunch-Num', 'Dinner-Num']), train.iloc[val_idx]['Lunch-Num']


param = {
        'max_depth': int(8.844),
        'colsample_bytree': 1.0,
        'colsample_bylevel': 1.0,
        'min_child_weight': 0.8725,
        'gamma': 0.0,
        'alpha': 0.6470988066711656,
        'lambda': 0.5556329807797603,
        'seed': 142,
    }

model = xgb.XGBRegressor(**param)
model.fit(train_X, train_Y, early_stopping_rounds=10,
                eval_metric='mae', eval_set=[(valid_X, valid_Y)], verbose=False)

pred_lunch.append(model.predict(test_X))
train_lunch.append(model.predict(train.drop(columns=['Lunch-Num', 'Dinner-Num'])))

kfolds = KFold(n_splits=10, shuffle=True, random_state=142)

pred_dinner = []
train_dinner = []

for n_fold, (trn_idx, val_idx) in enumerate(kfolds.split(train)):
    train_X, train_Y = train.iloc[trn_idx].drop(columns=['Lunch-Num', 'Dinner-Num']), train.iloc[trn_idx]['Dinner-Num']
    valid_X, valid_Y = train.iloc[val_idx].drop(columns=['Lunch-Num', 'Dinner-Num']), train.iloc[val_idx]['Dinner-Num']

    param = {
            'max_depth': int(9.562),
            'colsample_bytree': 0.9684,
            'colsample_bylevel': 0.9684,
            'min_child_weight': 0.5791,
            'gamma': 19.89,
            'alpha': 0.8186058572292882,
            'lambda': 0.40254700991884107,
            'seed': 142,

        }

    model = xgb.XGBRegressor(**param)
    model.fit(train_X, train_Y, early_stopping_rounds=10,
                    eval_metric='mae', eval_set=[(valid_X, valid_Y)], verbose=False)

    pred_dinner.append(model.predict(test_X))
    train_dinner.append(model.predict(train.drop(columns=['Lunch-Num', 'Dinner-Num'])))



submission['중식계'] = np.mean(pred_lunch, axis=0)+100


NameError: name 'train_X' is not defined

In [42]:
df = pd.read_csv('D:/csv/train.csv')
df.columns = ['date','day','numbers','dayoff','work','outsidework','workfhome','break','lunch','dinner','lunch_t','dinner_t']
dft = pd.read_csv('D:/csv/test.csv')
dft.columns = ['date','day','numbers','dayoff','work','outsidework','workfhome','break','lunch','dinner']


#결측데이터
day_miss = df[df['date'].str.contains('2018-06-01')].index
df.loc[day_miss[0], 'day'] = '금'

day_miss = df[df['date'].str.contains('2020-11-25')].index
df.loc[day_miss[0], 'dinner'] = '*'

#요일평균
df['avg_day_lunch'] = 0
df['avg_day_dinner'] = 0

dft['avg_day_lunch'] = 0
dft['avg_day_dinner'] = 0

def train_dayoftheweek_mean(i, j):
    meal = i[i['day'] == j]
    for idx in meal.index:
        i.loc[[idx], ['avg_day_lunch']] = meal['lunch_t'].mean()
        i.loc[[idx], ['avg_day_dinner']] = meal['dinner_t'].mean()

def test_dayoftheweek_mean(i, j):
    meal = df[df['day'] == j]
    test_idx = i[i['day'] == j].index
    for idx in test_idx:
        i.loc[[idx], ['avg_day_lunch']] = meal['lunch_t'].mean()
        i.loc[[idx], ['avg_day_dinner']] = meal['dinner_t'].mean()

for i in ['월', '화', '수', '목', '금']:
    train_dayoftheweek_mean(df, i)
    test_dayoftheweek_mean(dft, i)

#월별식수인원
df['avg_month_lunch'] = 0
df['avg_month_dinner'] = 0

dft['avg_month_lunch'] = 0
dft['avg_month_dinner'] = 0

month_01 = df[df['date'].astype(str).str.contains('-01-')]
month_02 = df[df['date'].astype(str).str.contains('-02-')]
month_03 = df[df['date'].astype(str).str.contains('-03-')]
month_04 = df[df['date'].astype(str).str.contains('-04-')]
month_05 = df[df['date'].astype(str).str.contains('-05-')]
month_06 = df[df['date'].astype(str).str.contains('-06-')]
month_07 = df[df['date'].astype(str).str.contains('-07-')]
month_08 = df[df['date'].astype(str).str.contains('-08-')]
month_09 = df[df['date'].astype(str).str.contains('-09-')]
month_10 = df[df['date'].astype(str).str.contains('-10-')]
month_11 = df[df['date'].astype(str).str.contains('-11-')]
month_12 = df[df['date'].astype(str).str.contains('-12-')]

train_month_total = [month_01, month_02, month_03, month_04, month_05, month_06,
                     month_07, month_08, month_09, month_10, month_11, month_12]

for month in train_month_total:
    for idx in month.index:
        df.loc[idx, 'avg_month_lunch'] = month['lunch_t'].mean()
        df.loc[idx, 'avg_month_dinner'] = month['dinner_t'].mean()

t_month_01 = dft[dft['date'].astype(str).str.contains('-01-')]
t_month_02 = dft[dft['date'].astype(str).str.contains('-02-')]
t_month_03 = dft[dft['date'].astype(str).str.contains('-03-')]
t_month_04 = dft[dft['date'].astype(str).str.contains('-04-')]

for idx in t_month_01.index:
    dft.loc[[idx], ['avg_month_lunch']] = month_01['lunch_t'].mean()
    dft.loc[[idx], ['avg_month_dinner']] = month_01['dinner_t'].mean()

for idx in t_month_02.index:
    dft.loc[[idx], ['avg_month_lunch']] = month_02['lunch_t'].mean()
    dft.loc[[idx], ['avg_month_dinner']] = month_02['dinner_t'].mean()

for idx in t_month_03.index:
    dft.loc[[idx], ['avg_month_lunch']] = month_03['lunch_t'].mean()
    dft.loc[[idx], ['avg_month_dinner']] = month_03['dinner_t'].mean()

for idx in t_month_04.index:
    dft.loc[[idx], ['avg_month_lunch']] = month_04['lunch_t'].mean()
    dft.loc[[idx], ['avg_month_dinner']] = month_04['dinner_t'].mean()

df['holiday'] = pd.DataFrame(index=range(0,len(df)), columns=['공휴일전후'])
df['holiday'] = df['holiday'].replace(np.nan ,0)

df._set_value(5, 'holiday', 2)
df._set_value(17, 'holiday', 1)
df._set_value(18, 'holiday', 2)
df._set_value(47, 'holiday', 1)
df._set_value(48, 'holiday', 2)
df._set_value(62, 'holiday', 1)
df._set_value(63, 'holiday', 2)
df._set_value(83, 'holiday', 2)
df._set_value(132, 'holiday', 2)
df._set_value(152, 'holiday', 1)
df._set_value(153, 'holiday', 2)
df._set_value(163, 'holiday', 2)
df._set_value(222, 'holiday', 2)
df._set_value(227, 'holiday', 2)
df._set_value(245, 'holiday', 1)
df._set_value(246, 'holiday', 2)
df._set_value(266, 'holiday', 1)
df._set_value(267, 'holiday', 2)
df._set_value(309, 'holiday', 1)
df._set_value(310, 'holiday', 3)
df._set_value(330, 'holiday', 1)
df._set_value(331, 'holiday', 2)
df._set_value(379, 'holiday', 1)
df._set_value(380, 'holiday', 2)
df._set_value(413, 'holiday', 2)
df._set_value(467, 'holiday', 2)
df._set_value(471, 'holiday', 2)
df._set_value(502, 'holiday', 1)
df._set_value(510, 'holiday', 1)
df._set_value(511, 'holiday', 2)
df._set_value(552, 'holiday', 1)
df._set_value(553, 'holiday', 2)
df._set_value(556, 'holiday', 2)
df._set_value(565, 'holiday', 1)
df._set_value(566, 'holiday', 2)
df._set_value(575, 'holiday', 1)
df._set_value(576, 'holiday', 2)
df._set_value(579, 'holiday', 1)
df._set_value(580, 'holiday', 2)
df._set_value(623, 'holiday', 1)
df._set_value(624, 'holiday', 2)
df._set_value(651, 'holiday', 1)
df._set_value(652, 'holiday', 2)
df._set_value(709, 'holiday', 1)
df._set_value(710, 'holiday', 2)
df._set_value(733, 'holiday', 2)
df._set_value(748, 'holiday', 1)
df._set_value(790, 'holiday', 1)
df._set_value(791, 'holiday', 2)
df._set_value(793, 'holiday', 2)
df._set_value(814, 'holiday', 1)
df._set_value(815, 'holiday', 2)
df._set_value(863, 'holiday', 1)
df._set_value(864, 'holiday', 2)
df._set_value(882, 'holiday', 1)
df._set_value(883, 'holiday', 2)
df._set_value(893, 'holiday', 1)
df._set_value(894, 'holiday', 3)
df._set_value(895, 'holiday', 2)
df._set_value(897, 'holiday', 1)
df._set_value(898, 'holiday', 2)
df._set_value(951, 'holiday', 1)
df._set_value(952, 'holiday', 2)
df._set_value(955, 'holiday', 1)
df._set_value(956, 'holiday', 2)
df._set_value(971, 'holiday', 1)
df._set_value(972, 'holiday', 2)
df._set_value(1027, 'holiday', 1)
df._set_value(1028, 'holiday', 2)
df._set_value(1037, 'holiday', 1)
df._set_value(1038, 'holiday', 3)
df._set_value(1039, 'holiday', 2)
df._set_value(1100, 'holiday', 2)
df._set_value(1129, 'holiday', 1)
df._set_value(1130, 'holiday', 2)
df._set_value(1133, 'holiday', 1)
df._set_value(1187, 'holiday', 1)
df._set_value(1188, 'holiday', 2)

In [43]:
#코로나 영향 <, 코로나로인한 회사내 재택근무 유무
covid = []
for i in range(len(df)):
    if i <= 1000:
        covid.append(0)
    else:
        covid.append(1)

df['covid'] = covid


dft['holiday'] = pd.DataFrame(index=range(0,len(dft)), columns=['공휴일전후'])
dft['holiday'] = dft['holiday'].replace(np.nan ,0)

dft._set_value(10, 'holiday', 1)
dft._set_value(21, 'holiday', 2)

covid = []
for i in range(len(dft)):
    covid.append(1)

dft['covid'] = covid


#저녁

dinner_menu = []
for menu in range(1068):  #1068  // len(df)
    slc = df.loc[menu,'dinner'].split(' ')
    slc = ' '.join(slc).split()

    for menu in slc:
        if '(' in menu:
            slc.remove(menu)
    dinner_menu.append(slc)

for menu in range(1068,len(df)):
    slc = df.loc[menu, 'dinner'].split(' ')
    slc = ' '.join(slc).split()
    dinner_menu.append(slc)


df['dinner_menu'] = dinner_menu

bobd = []
soupd = []
maind = []

for word in dinner_menu:

    if len(word) == 0:
        bobd.append('None')
        soupd.append('None')
        maind.append('None')
    elif '*' in word:
        bobd.append('None')
        soupd.append('None')
        maind.append('None')
    elif '가정의날' in word:
        bobd.append('None')
        soupd.append('None')
        maind.append('None')
    elif '가정의달' in word:
        bobd.append('None')
        soupd.append('None')
        maind.append('None')
    elif '자기계발의날' in word:
        bobd.append('None')
        soupd.append('None')
        maind.append('None')
    elif '*자기계발의날*' in word:
        bobd.append('None')
        soupd.append('None')
        maind.append('None')
    elif '자기개발의날' in word:
        bobd.append('None')
        soupd.append('None')
        maind.append('None')

    else:
        bobd.append(word[0])
        soupd.append(word[1])
        maind.append(word[2])

df['bobd'] = bobd
df['soupd'] = soupd
df['maind'] = maind

df['date'] = pd.to_datetime(df['date'])
df['yearmonth'] = df['date'].dt.strftime('%Y%m')
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%m%d')
df['Month'] = df['date'].str[0:2]
df['Date'] = df['date'].str[2:]
df.drop(['date','break'], inplace=True , axis=1)
df.drop(['yearmonth'],inplace=True , axis=1)
train = df[
    ['day','numbers','dayoff','work','outsidework','workfhome','dinner_t','Month','Date',
     'bobd','soupd','maind','avg_day_dinner','avg_month_dinner','covid','holiday']
]

train = train[~train['bobd'].str.contains("None", na=False, case=False)]

train['day'] =  train['day'].astype('category')
train['day'] = train.day.cat.codes

train.to_csv('2d.csv',index=False)

In [44]:
dft['date'] = pd.to_datetime(dft['date'])
dft['yearmonth'] = dft['date'].dt.strftime('%Y%m')
dft['date'] = pd.to_datetime(dft['date'])
dft['date'] = dft['date'].dt.strftime('%m%d')
dft.drop('yearmonth', inplace=True , axis=1)
dft['Month'] = dft['date'].str[0:2]
dft['Date'] = dft['date'].str[2:]
dft.drop(['break'], inplace=True , axis=1)


dinner_menu = []
for menu in range(len(dft)):  #1068  // len(df)
    slc = dft.loc[menu,'dinner'].split(' ')
    slc = ' '.join(slc).split()
    dinner_menu.append(slc)

dft['dinner_menu'] = dinner_menu

bobd=[]
soupd=[]
maind=[]

for i in range(len(dft)):
    if dinner_menu[i][0]:
        bobd.append(dinner_menu[i][0])
    if dinner_menu[i][1]:
        soupd.append(dinner_menu[i][1])
    if dinner_menu[i][2]:
        maind.append(dinner_menu[i][2])

dft['bobd'] = bobd
dft['soupd'] = soupd
dft['maind'] = maind

test = dft[
    ['day','numbers','dayoff','work','outsidework','workfhome','Month','Date','bobd','soupd','maind','avg_day_dinner','avg_month_dinner','covid','holiday']
]

test['day'] =  test['day'].astype('category')
test['day'] = test.day.cat.codes


test['bobd'] =  test['bobd'].astype('category')
test['bobd'] = test.bobd.cat.codes

test['soupd'] =  test['soupd'].astype('category')
test['soupd'] = test.soupd.cat.codes

test['maind'] =  test['maind'].astype('category')
test['maind'] = test.maind.cat.codes

test.to_csv('2dt.csv', index=False)

In [47]:
train = pd.read_csv('2d.csv')
test = pd.read_csv('2dt.csv')

x_train2 = train[['day', 'numbers', 'dayoff', 'work','outsidework', 'workfhome','Month','Date','avg_day_dinner','avg_month_dinner','holiday','covid']]
y2_train = train['dinner_t']
x_test2 = test[['day', 'numbers', 'dayoff', 'work','outsidework', 'workfhome','Month','Date','avg_day_dinner','avg_month_dinner','holiday','covid']]


model = xgb.XGBRegressor(max_depth=2,n_estimators=300,colsample_bylevel=0.5,colsample_bytree=0.5)


model.fit(x_train2, y2_train)

y_pred_dinner2 = model.predict(x_test2)
#결과 제출
submission['석식계'] = y_pred_dinner2

print(submission)
submission.to_csv('final2.csv', index=False)


            일자          중식계         석식계
0   2021-01-27  1112.602783  346.981781
1   2021-01-28  1067.120361  425.664429
2   2021-01-29   634.164429  239.263824
3   2021-02-01  1329.968872  556.008057
4   2021-02-02  1044.080566  466.364868
5   2021-02-03  1045.265503  453.826263
6   2021-02-04  1030.094849  459.820496
7   2021-02-05   757.361084  353.584686
8   2021-02-08  1427.650635  607.696777
9   2021-02-09  1077.423950  502.607208
10  2021-02-10  1018.731262  148.298859
11  2021-02-15  1399.591919  691.139099
12  2021-02-16  1165.594238  628.361694
13  2021-02-17  1065.123779  423.703033
14  2021-02-18   969.977417  485.181244
15  2021-02-19   689.033325  331.570465
16  2021-02-22  1436.693604  671.176025
17  2021-02-23  1185.643311  648.302063
18  2021-02-24   952.210632  351.161102
19  2021-02-25   961.202454  475.633209
20  2021-02-26   719.136414  232.304474
21  2021-03-02  1380.019775  723.425110
22  2021-03-03  1040.447388  479.444489
23  2021-03-04  1047.589600  549.106567
